## CÓDIGO IMDB ACCION

In [26]:
import pandas as pd

from bs4 import BeautifulSoup
import requests
import json
import csv
import random
import re

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep
import numpy as np

pd.set_option('display.max_columns', None)

In [28]:
with open ('../Fase1_datos/Extracciones/lista_accion.json', 'r') as fichero:
    lista_accion = json.load (fichero)

In [29]:
len(lista_accion)

3679

In [30]:
busqueda = lista_accion[3401:3421]

In [31]:
# SOLO EJECUTAR UNA VEZ
info_por_peli = []
fallos = []

In [32]:
# Acceder a la web mediente la Url y maximizar:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(random.randint(1,3))


# Rechazar cookies:
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(random.randint(1,3))

contador = 0

try:

    # Entrando en cada título
    for peli in busqueda:
        lista_info = [f'{peli[0]}',]
        contador +=1
        print(f'Película: {contador}')

        try:
            
            # Metemos el ID en el buscador
            try:
                driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
                sleep(random.randint(1,3))

                # Extraemos info
                info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
                lista_info.append(info.split('\n'))

                info_por_peli.append(lista_info)
            except:
                fallos.append(peli)
                driver.find_element('css selector', '#error > div.error_attrib > a').click()
                
                        
        except:
            
            continue

except requests.exceptions.RequestException as e:
    
    if isinstance(e, requests.exceptions.HTTPError) and e.response.status_code == 404:
            print('La página solicitada no fue encontrada (404).')
    else:
        print('Error al realizar la solicitud:', e)


        


Película: 1
Película: 2
Película: 3
Película: 4
Película: 5
Película: 6
Película: 7
Película: 8
Película: 9
Película: 10
Película: 11
Película: 12
Película: 13
Película: 14
Película: 15
Película: 16
Película: 17
Película: 18
Película: 19
Película: 20


In [8]:
# SOLO LA PRIMERA VEZ
lista_tuplas = []

In [18]:
# recogida de los datos
for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')
    patron_duracion = re.compile(r'\b(?:\d+h\s*)?\d+min\b')
    tupla_datos = [peli[0],] 

    # listas intermedias para ir checkeando
    direccion = []
    guion = []
    nota = []
    duracion = []
    argumento = []
        
    for indice, linea in enumerate(peli[1]):
               
        
        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            
            if r'\d+h' or r'\d+min' in peli[1][indice-1]:  
                duracion.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])

    # añadimos a la tupla los datos de la lista intermedia
    for elemento in [direccion, guion, nota, duracion, argumento]:
        if elemento != []:
            tupla_datos.append(elemento[0])
        else:
            tupla_datos.append(np.nan)
    
    lista_tuplas.append(tupla_datos)

In [19]:
# Creacion de Dataframe
df_tupla = pd.DataFrame(lista_tuplas)

In [20]:
# Guarda la informacion recopilada en un documento csv
df_tupla.to_csv('accion_IMDB.csv', index=False)

In [21]:
df_accion_IMDB = pd.read_csv("Archivos_IMDB/accion_IMDB.csv")

In [22]:
df_accion_limpio = df_accion_IMDB.drop_duplicates() 

In [23]:
df_accion_limpio.to_csv('Archivos_IMDB/accion_IMDB7.csv', index=False)